In [50]:
import requests
import time

def get_paging_results(url, data, pagesize = 40, maxpages = 10, sleep = .1):
    
    for page in range(1, maxpages + 1):
        
        response = requests.post(url, data = {
            "disciplineId": 10,
            "take": pagesize,
            "skip": (page - 1) * pagesize,
            "page": page,
            "pageSize": pagesize,
            **data
        })

        if len(response.json()['data']) == 0:
            break
        
        for result in response.json()['data']:
            yield result

        time.sleep(sleep)
            
def get_races(competition_id, **kwargs):
    
    url = "https://dataride.uci.ch/Results/iframe/Races/"
    data = {
        "competitionId": competition_id
    }
                             
    yield from get_paging_results(url, data, **kwargs)

def get_classifications(event_id, **kwargs):
    
    url = "https://dataride.uci.ch/Results/iframe/Results/"
    data = {
        "eventId": event_id
    }
    
    yield from get_paging_results(url, data, **kwargs)
    
def get_competitions(season_id, **kwargs):
    
    url = "https://dataride.uci.ch/Results/iframe/Competitions/"

    data = {
        "sort[0][field]": "StartDate",
        "sort[0][dir]": "desc",
        "filter[filters][0][field]": "RaceTypeId",
        "filter[filters][0][value]": 0,
        "filter[filters][1][field]": "CategoryId",
        "filter[filters][1][value]": 0,
        "filter[filters][2][field]": "SeasonId",
        "filter[filters][2][value]": season_id
    }

    yield from get_paging_results(url, data, **kwargs)
    
for comp in get_competitions(89, pagesize = 2, maxpages = 3):
    print(comp['CompetitionName'], comp["CompetitionId"])

Tour of Guangxi Women's WorldTour  50402
Japan Cup Cycle Road Race 50770
Gree-Tour of Guangxi 46172
Chrono des Nations 50595
Chrono des Nations 50661
Chrono des Nations 50790


In [40]:
for race in get_races(50402, pagesize = 2, maxpages = 2):
    print(race)

{'Id': 118843, 'RaceCode': 'F', 'Index': 1, 'StartDate': '/Date(1540072800000)/', 'EndDate': '/Date(1540072800000)/', 'RaceName': 'Final Classification', 'CategoryCode': 'Women Elite', 'RaceTypeCode': 'IRR', 'DisciplineCode': 'ROA', 'StartLocation': '', 'EndLocation': None, 'MandatoryDate': '/Date(1540072800000)/', 'EventResultPage': [], 'Venue': '', 'Date': '21 Oct 2018'}


In [41]:
for clf in get_classifications(118843, pagesize = 2, maxpages = 2):
    
    print(clf)

{'ResultId': 1958817, 'RankNumber': 1, 'SortOrder': 1, 'TeamType': 2, 'Rank': '1', 'Bib': None, 'MandatoryDate': '/Date(1347055200000)/', 'BirthDate': '/Date(564361200000)/', 'Age': '25', 'IndividualDisplayName': 'NUMAINVILLE Joelle', 'IndividualFirstName': None, 'IndividualLastName': None, 'UnknownIndividualDisplayName': None, 'UnknownIndividualFirstName': None, 'UnknownIndividualLastName': None, 'DisplayName': 'NUMAINVILLE Joelle', 'DisplayFirstName': None, 'DisplayLastName': None, 'IsoCode2': 'CA', 'NationName': 'CAN       ', 'IndividualCountryIsoCode2': 'CA', 'IndividualCountryName': 'CAN       ', 'IndividualCountryNameText': 'CANADA', 'UnknownIndividualCountryIsoCode2': None, 'UnknownIndividualCountryName': None, 'UnknownIndividualCountryNameText': None, 'TeamName': 'CANADA', 'ResultValue': '', 'PointPcR': None, 'TeamPointPcR': None, 'PcRPrefix': '', 'PcRSuffix': '', 'Irm': None, 'FlagCode': 'ca', 'Gender': '', 'IndividualGender': None, 'UnknownIndividualGender': None, 'Phase': No

In [53]:
def get_full_season(season_id):
    
    comps = list()
    for i, comp in enumerate(get_competitions(season_id, pagesize = 200, maxpages = 1000)):
        
        if(i % 50 == 0): print(f"Got comp {i} from season {season_id}, {comp['CompetitionName']}")
        comps.append(comp)                     
            
    return comps

SEASON_IDS = {
    2018: 89,
    2017: 19,
    2016: 15,
    2015: 7,
    2014: 85,
    2013: 84,
    2012: 83,
    2011: 82,
    2010: 81,
    2009: 80
}

def get_all_seasons():
    
    comps = list()                           
    for sid in SEASON_IDS.values():
        comps += get_full_season(sid)
    
    return comps
    
comps = get_all_seasons()

Got comp 0 from season 89, Tour of Guangxi Women's WorldTour 
Got comp 50 from season 89, Giro della Toscana - Memorial Alfredo Martini
Got comp 100 from season 89, La Vuelta ciclista a España
Got comp 150 from season 89, Aubel -Thimister - Stavelot
Got comp 200 from season 89, Championnats Nationaux Route - Algérie
Got comp 250 from season 89, National Road Championships - Kazakhstan (WE,WJ)
Got comp 300 from season 89, GP Horsens Posten
Got comp 350 from season 89, Horizon Park Women Challenge
Got comp 400 from season 89, Amgen Tour of California
Got comp 450 from season 89, Festival Elsy Jacobs
Got comp 500 from season 89, The Princess Maha Chackri Sirindhorn's  Cup "Women's Tour of Thailand"
Got comp 550 from season 89, Volta Ciclista a Catalunya
Got comp 600 from season 89, GP Laguna Porec
Got comp 650 from season 89, Tour of Hainan
Got comp 0 from season 19, Tour of Guangxi Women’s Elite World Challenge
Got comp 50 from season 19, Grand Prix d'Isbergues - Pas de Calais
Got comp 1

In [56]:
import pandas as pd

comp_df = pd.DataFrame(comps)
comp_df.sample(5)

,ClassCode,CompetitionId,CompetitionName,CountryIsoCode2,CountryIsoCode3,CountryName,Date,EndDate,FlagCode,IsDone,IsInProgress,StartDate
4728,1.1,44563,Chrono des Nations,FR,FRA,FRANCE,18 Oct 2009,/Date(1255816800000)/,fr,False,False,/Date(1255816800000)/
3555,1.1,42729,Ronde Van Zeeland Seaports,NL,NED,NETHERLANDS,09 Jun 2012,/Date(1339192800000)/,nl,False,False,/Date(1339192800000)/
2519,1.1,43958,Tour du Doubs - Conseil Général,FR,FRA,FRANCE,14 Sep 2014,/Date(1410645600000)/,fr,False,False,/Date(1410645600000)/
3748,1.1,45661,Grand Prix GSB,SV,ESA,EL SALVADOR,21 Mar 2012,/Date(1332284400000)/,sv,False,False,/Date(1332284400000)/
2767,2.HC,43821,Presidential Cycling Tour of Turkey,TR,TUR,TURKEY,27 Apr-04 May 2014,/Date(1399154400000)/,tr,False,False,/Date(1398549600000)/


In [64]:
import json

json.dump(comps, open("./all_comps.json", "w"))

In [70]:
races = list()

for i, comp in enumerate(comps):
    for n, race in enumerate(get_races(comp["CompetitionId"], pagesize = 200, maxpages = 10)):
        races.append({
            "CompetitionId": comp["CompetitionId"], 
            **race
        })
    
    print(f"Got {n} classifications from comp {comp['CompetitionName']}")

Got 0 classifications from comp Tour of Guangxi Women's WorldTour 
Got 0 classifications from comp Japan Cup Cycle Road Race
Got 6 classifications from comp Gree-Tour of Guangxi
Got 0 classifications from comp Chrono des Nations
Got 0 classifications from comp Chrono des Nations
Got 0 classifications from comp Chrono des Nations
Got 0 classifications from comp Oita Urban Classic
Got 0 classifications from comp Chrono des Nations
Got 0 classifications from comp Chrono des Nations
Got 0 classifications from comp Hammer Hong Kong
Got 5 classifications from comp Jeux Olympiques de la Jeunesse / Youth Olympic Games
Got 0 classifications from comp Il Lombardia
Got 0 classifications from comp De Kustpijl
Got 0 classifications from comp Tacx Pro Classic / Ronde van Zeeland
Got 4 classifications from comp Elite Road Caribbean Championships
Got 0 classifications from comp Gran Piemonte
Got 0 classifications from comp Milano-Torino
Got 5 classifications from comp Vuelta a Colombia Femenina
Got 6 

Got 0 classifications from comp KOGA Slag om Norg
Got 3 classifications from comp Campeonatos Panamericanos Junior
Got 4 classifications from comp Czech Cycling Tour
Got 6 classifications from comp Cycling Tour of Szeklerland
Got 5 classifications from comp Vuelta a Burgos
Got 7 classifications from comp The Larry H.Miller Tour of Utah
Got 8 classifications from comp Tour du Rwanda
Got 0 classifications from comp La Poly Normande
Got 3 classifications from comp UEC Road Elite European Championships
Got 0 classifications from comp Antwerpse Havenpijl
Got 0 classifications from comp Clasica Ciclista San Sebastian
Got 7 classifications from comp Tour de Pologne
Got 0 classifications from comp Erondegemse Pijl (Erpe-Mere)
Got 0 classifications from comp Hansa Bygg Kalmar Grand Prix Road Race
Got 4 classifications from comp Aubel -Thimister - Stavelot
Got 11 classifications from comp Tour Cycliste International de la Guadeloupe
Got 3 classifications from comp 1° Giro del Nordest d'Italia
G

Got 7 classifications from comp National Road Championships - Guatemala
Got 2 classifications from comp National Road Championships (IRR) - Slovenia 
Got 0 classifications from comp National Road Championships (IRR) - Slovenia (WE)
Got 9 classifications from comp National Road Championships - USA
Got 7 classifications from comp National Road Championships - Belarus
Got 7 classifications from comp National Road Championships - Croatia
Got 3 classifications from comp National Road Championships - Rwanda (ME,MU,MJ)
Got 1 classifications from comp National Road Championships - Croatia (WE,WU,WJ)
Got 9 classifications from comp National Road Championships - Netherlands
Got 6 classifications from comp National Road Championships - Belize
Got 11 classifications from comp National Road Championships - Israel 
Got 1 classifications from comp National Road Championships - Montenegro
Got 1 classifications from comp National Road Championships - Korea (ME)
Got 3 classifications from comp National 

Got 2 classifications from comp Tour of Estonia
Got 3 classifications from comp Hammer Stavanger
Got 5 classifications from comp Tour du Pays de Vaud
Got 3 classifications from comp Gemenc Grand Prix
Got 5 classifications from comp Baloise Belgium Tour
Got 3 classifications from comp Tour des Fjords
Got 8 classifications from comp Tour of Japan
Got 8 classifications from comp Rás Tailteann
Got 0 classifications from comp Grote Prijs Marcel Kint
Got 4 classifications from comp Le Tour de Filipinas
Got 4 classifications from comp WWT Emakumeen XXXI. Bira
Got 3 classifications from comp Trophée Centre Morbihan
Got 5 classifications from comp 75th Tour of Albania
Got 4 classifications from comp 36e SPIE Internationale Juniorendriedaagse
Got 3 classifications from comp A Travers Les Hauts De France - Trophée Paris-Arras Tour
Got 3 classifications from comp Tour de l'Ain
Got 3 classifications from comp Amgen Tour of California Women’s Race empowered with SRAM
Got 0 classifications from comp 

Got 0 classifications from comp Grand Prix Bob Jungels
Got 6 classifications from comp The Princess Maha Chackri Sirindhorn's  Cup "Tour of Thailand"
Got 0 classifications from comp 70° Trofeo Piva
Got 0 classifications from comp GP Adria Mobil
Got 0 classifications from comp La Roue Tourangelle Région Centre Val de Loire - Trophée Harmonie Mutuelle
Got 4 classifications from comp Le Triptyque des Monts et Châteaux
Got 0 classifications from comp Visegrad 4 Bicycle Race- GP Slovakia
Got 0 classifications from comp Volta Limburg Classic
Got 0 classifications from comp Gran Premio Miguel Indurain
Got 0 classifications from comp Route Adélie de Vitré
Got 2 classifications from comp Tour of Fatih Sultan Mehmet
Got 0 classifications from comp Dwars door Vlaanderen - A travers la Flandre
Got 0 classifications from comp Dwars door Vlaanderen / A travers la Flandre
Got 7 classifications from comp Tour d'Algérie Cycliste
Got 0 classifications from comp Gent-Wevelgem in Flanders Fields
Got 0 cla

...

Got 6 classifications from comp Internationale Thüringen-Rundfahrt U23
Got 6 classifications from comp Tour de Serbie
Got 0 classifications from comp Val d'Ille U Classic 35
Got 0 classifications from comp Polygon Tour de Jakarta
Got 0 classifications from comp Grand Prix Cycliste de Gatineau
Got 3 classifications from comp Delta Tour Zeeland
Got 9 classifications from comp Giro Ciclistico d'Italia
Got 3 classifications from comp Oberösterreichrundfahrt
Got 5 classifications from comp Iurreta-Emakumeen Bira
Got 4 classifications from comp GP PAD
Got 4 classifications from comp Ronde de l'Oise
Got 8 classifications from comp Circuito Montañés
Got 0 classifications from comp Durango-Durango Emakumeen Saria
Got 0 classifications from comp GP du canton d'Argovie
Got 0 classifications from comp Memorial Ph. Van Coningsloo
Got 0 classifications from comp Coppa della Pace - Trofeo F.lli Anelli
Got 0 classifications from comp Philadelphia International Championship
Got 0 classifications from c

Got 0 classifications from comp Trofeo Edil C
Got 0 classifications from comp Albert Achterhes Profronde van Drenthe
Got 4 classifications from comp Circuit des Ardennes International
Got 0 classifications from comp Grand Prix Pino Cerami
Got 4 classifications from comp Cinturón Ciclista Internacional a Mallorca
Got 0 classifications from comp Drentse 8
Got 0 classifications from comp Scheldeprijs
Got 5 classifications from comp Circuit Cycliste Sarthe - Pays de la Loire
Got 0 classifications from comp GP Palio del Recioto
Got 0 classifications from comp Rund um Köln
Got 0 classifications from comp Giro del Belvedere
Got 0 classifications from comp Grand Prix de Dottignies
Got 0 classifications from comp Trofeo Piva Banca Popolare di Vicenza
Got 0 classifications from comp Grand Prix de la Ville de Nogent-sur-Oise
Got 0 classifications from comp Ronde van Vlaanderen / Tour des Flandres
Got 0 classifications from comp Hel van het Mergelland
Got 0 classifications from comp Gran Premio Mi

Got 0 classifications from comp Praha - Karlovy Vary - Praha
Got 0 classifications from comp Grand Prix de la Somme « Conseil Général 80»
Got 3 classifications from comp Kroz Istru / Tour d'Istrie
Got 0 classifications from comp Kampioenschap van Vlaanderen
Got 0 classifications from comp Tour of Vojvodina I part
Got 0 classifications from comp Grand Prix de Wallonie
Got 6 classifications from comp Giro della Toscana Int. Femminile - Memorial Michela Fanini
Got 0 classifications from comp GP de Fourmies / La Voix du Nord
Got 0 classifications from comp Parijs-Brusssel - Paris-Bruxelles
Got 8 classifications from comp Tour of  Britain
Got 2 classifications from comp Univest Grand Prix
Got 0 classifications from comp Memorial Rik Van Steenbergen
Got 6 classifications from comp Tour de Hokkaido
Got 6 classifications from comp Tour Cycliste Féminin International Ardèche
Got 7 classifications from comp Tour of Missouri
Got 0 classifications from comp Giro della Romagna
Got 0 classifications

Got 4 classifications from comp Boucles de la Mayenne
Got 3 classifications from comp Tour of Malopolska
Got 3 classifications from comp Rabo Ster Zeeuwsche Eilanden
Got 4 classifications from comp Grande Boucle Féminine Internationale
Got 5 classifications from comp Ster Elektrotoer
Got 7 classifications from comp Tour de Serbie
Got 3 classifications from comp Delta Tour Zeeland
Got 9 classifications from comp Giro Ciclistico d'Italia
Got 5 classifications from comp Trofeo Karlsberg
Got 5 classifications from comp Iurreta-Emakumeen Bira
Got 4 classifications from comp GP Internacional CTT Correios de Portugal
Got 4 classifications from comp Ronde de l'Oise
Got 5 classifications from comp Tour de Lorraine Juniors
Got 7 classifications from comp Circuito Montañés
Got 6 classifications from comp Tour de Beauce
Got 0 classifications from comp Durango-Durango Emakumeen Saria
Got 0 classifications from comp GP du canton d'Argovie, Gippingen
Got 7 classifications from comp Internationale Thü

Got 0 classifications from comp Paris-Camembert
Got 6 classifications from comp Tour du Loir-et-Cher 'E. Provost'
Got 0 classifications from comp GP Palio del Recioto
Got 0 classifications from comp Rund um Köln
Got 0 classifications from comp Giro del Belvedere di Villa di Cordignano
Got 0 classifications from comp Unive Ronde van Drenthe
Got 0 classifications from comp Albert Achterhes Profronde van Drenthe
Got 0 classifications from comp Paris - Roubaix
Got 0 classifications from comp Klasika Primavera
Got 8 classifications from comp Presidential Cycling Tour
Got 0 classifications from comp Paris - Roubaix Juniors
Got 0 classifications from comp Grand Prix of Donetsk
Got 0 classifications from comp Novilon Eurocup Ronde van Drenthe
Got 10 classifications from comp Tour du Maroc
Got 4 classifications from comp Circuit des Ardennes International
Got 0 classifications from comp Drentse 8 van Dwingeloo
Got 5 classifications from comp Circuit Cycliste Sarthe - Pays de la Loire
Got 0 clas

In [71]:
len(races)

17836

In [74]:
json.dump(races, open("./all_races.json", "w"))

In [75]:
%ls -lha

total 8,8M
drwxrwxr-x  5 gijs gijs 4,0K Nov 23 13:10 ./
drwxr-xr-x 26 gijs gijs 4,0K Sep 15 15:34 ../
-rw-rw-r--  1 gijs gijs 1,7M Nov 23 12:07 all_comps.json
-rw-rw-r--  1 gijs gijs 6,7M Nov 23 13:10 all_races.json
drwxrwxr-x  8 gijs gijs 4,0K Nov 23 10:38 .git/
-rw-rw-r--  1 gijs gijs   32 Jun 18 23:33 .gitignore
drwxrwxr-x  2 gijs gijs 4,0K Jun 18 23:15 .ipynb_checkpoints/
drwxrwxr-x  2 gijs gijs 4,0K Jun 18 22:26 __pycache__/
-rw-rw-r--  1 gijs gijs   50 Jun 18 23:33 README.md
-rw-rw-r--  1 gijs gijs 386K Nov 23 13:05 uci_data_scraping.ipynb


In [80]:
clfs = list()


for i, race in enumerate(races):
    for clf in get_classifications(race["Id"], maxpages = 10, pagesize = 250):
        clfs.append({
            "RaceId": race["Id"],
            **clf
        })
    
    if i % 50 == 0:
        
        print(f"Now at {i} races, got {len(clfs)} classifications total.")

Now at 0 races, got 1 classifications total.
Now at 50 races, got 3021 classifications total.
Now at 100 races, got 5267 classifications total.
Now at 150 races, got 7460 classifications total.
Now at 200 races, got 9569 classifications total.
Now at 250 races, got 11756 classifications total.
Now at 300 races, got 14199 classifications total.
Now at 350 races, got 16743 classifications total.
Now at 400 races, got 19518 classifications total.
Now at 450 races, got 22321 classifications total.
Now at 500 races, got 24290 classifications total.
Now at 550 races, got 27185 classifications total.
Now at 600 races, got 30032 classifications total.
Now at 650 races, got 31656 classifications total.
Now at 700 races, got 33681 classifications total.
Now at 750 races, got 36037 classifications total.
Now at 800 races, got 39211 classifications total.
Now at 850 races, got 42593 classifications total.
Now at 900 races, got 45559 classifications total.
Now at 950 races, got 48037 classification

Now at 7800 races, got 197404 classifications total.
Now at 7850 races, got 199660 classifications total.
Now at 7900 races, got 202198 classifications total.
Now at 7950 races, got 204057 classifications total.
Now at 8000 races, got 206931 classifications total.
Now at 8050 races, got 208200 classifications total.
Now at 8100 races, got 210317 classifications total.
Now at 8150 races, got 213068 classifications total.
Now at 8200 races, got 215132 classifications total.
Now at 8250 races, got 217341 classifications total.
Now at 8300 races, got 219586 classifications total.
Now at 8350 races, got 221984 classifications total.
Now at 8400 races, got 223809 classifications total.
Now at 8450 races, got 226510 classifications total.
Now at 8500 races, got 227693 classifications total.
Now at 8550 races, got 229821 classifications total.
Now at 8600 races, got 231548 classifications total.
Now at 8650 races, got 232530 classifications total.
Now at 8700 races, got 235080 classifications 

Now at 15450 races, got 521954 classifications total.
Now at 15500 races, got 523896 classifications total.
Now at 15550 races, got 526651 classifications total.
Now at 15600 races, got 530033 classifications total.
Now at 15650 races, got 532815 classifications total.
Now at 15700 races, got 535053 classifications total.
Now at 15750 races, got 537113 classifications total.
Now at 15800 races, got 539593 classifications total.
Now at 15850 races, got 543080 classifications total.
Now at 15900 races, got 545270 classifications total.
Now at 15950 races, got 547527 classifications total.
Now at 16000 races, got 550396 classifications total.
Now at 16050 races, got 552351 classifications total.
Now at 16100 races, got 554499 classifications total.
Now at 16150 races, got 556620 classifications total.
Now at 16200 races, got 559515 classifications total.
Now at 16250 races, got 561290 classifications total.
Now at 16300 races, got 563207 classifications total.
Now at 16350 races, got 5650

In [78]:
clfs

[{'RaceId': 118843,
  'ResultId': 1958817,
  'RankNumber': 1,
  'SortOrder': 1,
  'TeamType': 2,
  'Rank': '1',
  'Bib': None,
  'MandatoryDate': '/Date(1347055200000)/',
  'BirthDate': '/Date(564361200000)/',
  'Age': '25',
  'IndividualDisplayName': 'NUMAINVILLE Joelle',
  'IndividualFirstName': None,
  'IndividualLastName': None,
  'UnknownIndividualDisplayName': None,
  'UnknownIndividualFirstName': None,
  'UnknownIndividualLastName': None,
  'DisplayName': 'NUMAINVILLE Joelle',
  'DisplayFirstName': None,
  'DisplayLastName': None,
  'IsoCode2': 'CA',
  'NationName': 'CAN       ',
  'IndividualCountryIsoCode2': 'CA',
  'IndividualCountryName': 'CAN       ',
  'IndividualCountryNameText': 'CANADA',
  'UnknownIndividualCountryIsoCode2': None,
  'UnknownIndividualCountryName': None,
  'UnknownIndividualCountryNameText': None,
  'TeamName': 'CANADA',
  'ResultValue': '',
  'PointPcR': None,
  'TeamPointPcR': None,
  'PcRPrefix': '',
  'PcRSuffix': '',
  'Irm': None,
  'FlagCode': 'ca

In [81]:
len(clfs)

645649

In [82]:
json.dump(clfs, open("all_clfs.json", "w"))

In [83]:
%ls -lha ./all_clfs.json

-rw-rw-r-- 1 gijs gijs 647M Nov 23 17:40 ./all_clfs.json
